# Chapter 4: Creating Data

Install the following package:

```bash
pip install nbformat
```

Run the table definition program from the previous chapter:

In [38]:
%run -i 'ch03.ipynb'

2024-03-20 08:06:56,016 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-03-20 08:06:56,017 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("employee")
2024-03-20 08:06:56,018 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-03-20 08:06:56,019 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("employee")
2024-03-20 08:06:56,019 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-03-20 08:06:56,020 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("product")
2024-03-20 08:06:56,021 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-03-20 08:06:56,022 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("product")
2024-03-20 08:06:56,022 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-03-20 08:06:56,023 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("customer")
2024-03-20 08:06:56,023 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-03-20 08:06:56,024 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("customer")
2024-03-20 08:06:56,025 INFO sqlalchemy.engine.Engine [raw sql

Get the connection:

In [39]:
conn = engine.connect()

Necessary imports:

In [40]:
from datetime import date, timedelta
from sqlalchemy import insert

Let's start inserting employee records:

In [41]:
employee_ids = []

In [42]:
stmt = (
    insert(employee)
    .values(
        name="Alice",
        is_manager=True,
        hire_date=(date.today() - timedelta(days=1)),
    )
)

In [43]:
print(stmt)

INSERT INTO employee (name, is_manager, hire_date) VALUES (:name, :is_manager, :hire_date)


In [44]:
compiled = stmt.compile()

In [45]:
print(compiled.params)

{'name': 'Alice', 'is_manager': True, 'hire_date': datetime.date(2024, 3, 19)}


In [46]:
result = conn.execute(stmt)

2024-03-20 08:06:56,110 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-03-20 08:06:56,110 INFO sqlalchemy.engine.Engine INSERT INTO employee (name, is_manager, hire_date) VALUES (?, ?, ?)
2024-03-20 08:06:56,111 INFO sqlalchemy.engine.Engine [generated in 0.00104s] ('Alice', 1, '2024-03-19')


In [47]:
print(result.inserted_primary_key)

(1,)


In [48]:
manager_alice_id = result.inserted_primary_key[0]

In [49]:
employee_ids.append(manager_alice_id)

In [50]:
conn.commit()

2024-03-20 08:06:56,126 INFO sqlalchemy.engine.Engine COMMIT


Using the RETURNING clause:

In [51]:
result = conn.execute(
    (
        insert(employee)
        .returning(
            employee.c.employee_id,
            employee.c.name,
            employee.c.hire_date,
        )
    ),
    [
        {"manager_id": manager_alice_id, "name": "Bob"},
        {"manager_id": manager_alice_id, "name": "Cathy"},
    ],
)

2024-03-20 08:06:56,133 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-03-20 08:06:56,134 INFO sqlalchemy.engine.Engine INSERT INTO employee (manager_id, name, is_manager, hire_date) VALUES (?, ?, ?, ?), (?, ?, ?, ?) RETURNING employee_id, name, hire_date
2024-03-20 08:06:56,134 INFO sqlalchemy.engine.Engine [generated in 0.00081s (insertmanyvalues) 1/1 (unordered)] (1, 'Bob', 0, '2024-03-20', 1, 'Cathy', 0, '2024-03-20')


In [52]:
print(result.rowcount)

2


In [53]:
print(list(result.keys()))

['employee_id', 'name', 'hire_date']


In [54]:
for row in result:
    print(row)
    employee_ids.append(row.employee_id)

(2, 'Bob', datetime.date(2024, 3, 20))
(3, 'Cathy', datetime.date(2024, 3, 20))


In [55]:
conn.commit()

2024-03-20 08:06:56,150 INFO sqlalchemy.engine.Engine COMMIT


More employees:

In [56]:
result = conn.execute(
    insert(employee)
    .values(
        name="Louis",
        is_manager=True,
        hire_date=(date.today() - timedelta(days=30)),
    )
)
manager_louis_id = result.inserted_primary_key[0]
employee_ids.append(manager_louis_id)

result = conn.execute(
    insert(employee)
    .values(
        name="Lilly",
        manager_id=manager_louis_id,
        hire_date=(date.today() - timedelta(days=20)),
    )
)
employee_ids.append(result.inserted_primary_key[0])

conn.commit()  # commit as you go

conn.execute(
    insert(employee).values(
        name="Alice",
    )
)
conn.commit()  # commit as you go

2024-03-20 08:06:56,157 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-03-20 08:06:56,158 INFO sqlalchemy.engine.Engine INSERT INTO employee (name, is_manager, hire_date) VALUES (?, ?, ?)
2024-03-20 08:06:56,158 INFO sqlalchemy.engine.Engine [cached since 0.04856s ago] ('Louis', 1, '2024-02-19')
2024-03-20 08:06:56,159 INFO sqlalchemy.engine.Engine INSERT INTO employee (manager_id, name, is_manager, hire_date) VALUES (?, ?, ?, ?)
2024-03-20 08:06:56,160 INFO sqlalchemy.engine.Engine [generated in 0.00034s] (4, 'Lilly', 0, '2024-02-29')
2024-03-20 08:06:56,160 INFO sqlalchemy.engine.Engine COMMIT
2024-03-20 08:06:56,162 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-03-20 08:06:56,163 INFO sqlalchemy.engine.Engine INSERT INTO employee (name, is_manager, hire_date) VALUES (?, ?, ?)
2024-03-20 08:06:56,163 INFO sqlalchemy.engine.Engine [generated in 0.00066s] ('Alice', 0, '2024-03-20')
2024-03-20 08:06:56,164 INFO sqlalchemy.engine.Engine COMMIT


Employee PKs are saved for later use:

In [57]:
print(employee_ids)

[1, 2, 3, 4, 5]


Input product data:

In [58]:
product_data = [
    {
        "product_name": "phone",
        "unit_price": 300.0,
        "units_in_stock": 5,
        "type": ProductType.PHONE,
    },
    {
        "product_name": "phone screen protector",
        "unit_price": 9.50,
        "units_in_stock": 10,
        "type": ProductType.ACCESSORY,
    },
    {
        "product_name": "headphone",
        "unit_price": 25.99,
        "units_in_stock": 10,
        "type": ProductType.ACCESSORY,
    },
    {
        "product_name": "digital camera",
        "unit_price": 45.99,
        "units_in_stock": 5,
        "type": ProductType.OTHER,
    },
    {
        "product_name": "memory card 256GB",
        "unit_price": 21.99,
        "units_in_stock": 1,
        "type": ProductType.ACCESSORY,
    },
]

In [59]:
product_ids = []

In [60]:
result = conn.execute(
    insert(product).returning(
        # c: collection of table columns
        product.c.product_id, product.c.product_name
    ),
    product_data,
)

2024-03-20 08:06:56,182 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-03-20 08:06:56,183 INFO sqlalchemy.engine.Engine INSERT INTO product (product_name, unit_price, units_in_stock, type) VALUES (?, ?, ?, ?), (?, ?, ?, ?), (?, ?, ?, ?), (?, ?, ?, ?), (?, ?, ?, ?) RETURNING product_id, product_name
2024-03-20 08:06:56,184 INFO sqlalchemy.engine.Engine [generated in 0.00079s (insertmanyvalues) 1/1 (unordered)] ('phone', 300.0, 5, 'PHONE', 'phone screen protector', 9.5, 10, 'ACCESSORY', 'headphone', 25.99, 10, 'ACCESSORY', 'digital camera', 45.99, 5, 'OTHER', 'memory card 256GB', 21.99, 1, 'ACCESSORY')


In [61]:
for row in result:
    print(row)  # (product_id, product_name)
    product_ids.append(row[0])

(1, 'phone')
(2, 'phone screen protector')
(3, 'headphone')
(4, 'digital camera')
(5, 'memory card 256GB')


In [62]:
conn.commit()

2024-03-20 08:06:56,192 INFO sqlalchemy.engine.Engine COMMIT


Product PKs for later use:

In [63]:
print(product_ids)

[1, 2, 3, 4, 5]


Input customer data:

In [64]:
customer_data = [
    {
        "first_name": "Alex",
        "last_name": "Smith",
        "address": "618 Oak Lane, CA",
        "email": "alex_smith@test.com",
    },
    {
        "first_name": "Mary",
        "last_name": "Taylor",
        "address": "200-139 Jefferson Street, NY",
        "email": "mary_taylor@test.com",
    },
]

In [65]:
customer_ids = []

In [66]:
try:
    result = conn.execute(
        (
            insert(customer)
            .returning(
                customer.c.customer_id,
                customer.c.first_name,
            )
        ),
        customer_data
    )
    for row in result:
        print(row)  # row contains tuples of (customer_id, first_name)
        customer_ids.append(row[0])
except IntegrityError as e:
    print("Error creating customers!", e)

2024-03-20 08:06:56,214 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-03-20 08:06:56,214 INFO sqlalchemy.engine.Engine INSERT INTO customer (first_name, last_name, address, email) VALUES (?, ?, ?, ?), (?, ?, ?, ?) RETURNING customer_id, first_name
2024-03-20 08:06:56,215 INFO sqlalchemy.engine.Engine [generated in 0.00076s (insertmanyvalues) 1/1 (unordered)] ('Alex', 'Smith', '618 Oak Lane, CA', 'alex_smith@test.com', 'Mary', 'Taylor', '200-139 Jefferson Street, NY', 'mary_taylor@test.com')
(1, 'Alex')
(2, 'Mary')


Customer PKs:

In [67]:
print(customer_ids)

[1, 2]


In [68]:
conn.commit()

2024-03-20 08:06:56,224 INFO sqlalchemy.engine.Engine COMMIT


Customer and product PKs from previous return values:

In [69]:
c1, c2 = customer_ids
p1, p2, p3, p4, p5 = product_ids

Input order and order details:

In [70]:
order_ids = []

First customer places 1st order:

In [71]:
result = conn.execute(insert(order).values(customer_id=c1))
order1_id = result.inserted_primary_key[0]
order_ids.append(order1_id)
result = conn.execute(
    insert(order_detail),
    [
        # phone
        {"order_id": order1_id, "product_id": p1, "quantity": 1},
        # phone screen protector
        {"order_id": order1_id, "product_id": p2, "quantity": 1},
        # headphone
        {"order_id": order1_id, "product_id": p3, "quantity": 1},
    ]
)
print("order_detail PKs:", result.inserted_primary_key_rows)
conn.commit()  # commit as you go

2024-03-20 08:06:56,242 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-03-20 08:06:56,243 INFO sqlalchemy.engine.Engine INSERT INTO "order" (customer_id, order_datetime, is_shipped) VALUES (?, ?, ?)
2024-03-20 08:06:56,243 INFO sqlalchemy.engine.Engine [generated in 0.00116s] (1, '2024-03-20 08:06:56.242654', 0)
2024-03-20 08:06:56,244 INFO sqlalchemy.engine.Engine INSERT INTO order_detail (order_id, product_id, quantity) VALUES (?, ?, ?)
2024-03-20 08:06:56,245 INFO sqlalchemy.engine.Engine [generated in 0.00036s] [(1, 1, 1), (1, 2, 1), (1, 3, 1)]
order_detail PKs: [(1, 1), (1, 2), (1, 3)]
2024-03-20 08:06:56,246 INFO sqlalchemy.engine.Engine COMMIT


First customer places 2nd order:

In [72]:
result = conn.execute(insert(order).values(customer_id=c1))
order2_id = result.inserted_primary_key[0]
order_ids.append(order2_id)
conn.execute(
    insert(order_detail)
    .values(
        order_id=order2_id,
        product_id=p5,  # memory card
        quantity=1,
    )
)
conn.commit()  # commit as you go

2024-03-20 08:06:56,251 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-03-20 08:06:56,252 INFO sqlalchemy.engine.Engine INSERT INTO "order" (customer_id, order_datetime, is_shipped) VALUES (?, ?, ?)
2024-03-20 08:06:56,252 INFO sqlalchemy.engine.Engine [cached since 0.01023s ago] (1, '2024-03-20 08:06:56.251933', 0)
2024-03-20 08:06:56,254 INFO sqlalchemy.engine.Engine INSERT INTO order_detail (order_id, product_id, quantity) VALUES (?, ?, ?)
2024-03-20 08:06:56,254 INFO sqlalchemy.engine.Engine [generated in 0.00035s] (2, 5, 1)
2024-03-20 08:06:56,255 INFO sqlalchemy.engine.Engine COMMIT


Second customer places order on camera related products:

In [73]:
result = conn.execute(insert(order).values(customer_id=c2))
order3_id = result.inserted_primary_key[0]
order_ids.append(order3_id)
conn.execute(
    insert(order_detail),
    [
        # digital camera
        {"order_id": order3_id, "product_id": p4, "quantity": 1},
        # memory card (won't be enough for all!)
        {"order_id": order3_id, "product_id": p5, "quantity": 2},
    ]
)
conn.commit()  # commit as you go

2024-03-20 08:06:56,261 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-03-20 08:06:56,261 INFO sqlalchemy.engine.Engine INSERT INTO "order" (customer_id, order_datetime, is_shipped) VALUES (?, ?, ?)
2024-03-20 08:06:56,262 INFO sqlalchemy.engine.Engine [cached since 0.01955s ago] (2, '2024-03-20 08:06:56.261256', 0)
2024-03-20 08:06:56,263 INFO sqlalchemy.engine.Engine INSERT INTO order_detail (order_id, product_id, quantity) VALUES (?, ?, ?)
2024-03-20 08:06:56,263 INFO sqlalchemy.engine.Engine [cached since 0.01847s ago] [(3, 4, 1), (3, 5, 2)]
2024-03-20 08:06:56,263 INFO sqlalchemy.engine.Engine COMMIT


Remember to close the connection when you're done:

In [74]:
conn.close()